In [1]:
using BenchmarkTools

In [2]:
function Convolution_2d(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [3]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [4]:
function Conv_backward_v2( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] += Convolution_2d(tmp_weights, tmp_gradient; padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [5]:
input = rand(Float32, 28, 28, 6);
weights = rand(Float32, 3, 3, 6, 16);
bias = rand(Float32, 16);
gradient = rand(Float32, 26, 26, 16);

In [6]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 71 samples with 1 evaluation.
 Range (min … max):  56.583 ms … 78.388 ms  ┊ GC (min … max): 28.78% … 35.06%
 Time  (median):     71.070 ms              ┊ GC (median):    28.98%
 Time  (mean ± σ):   70.668 ms ±  3.250 ms  ┊ GC (mean ± σ):  29.23% ±  1.33%

                                      █▃         ▃▂▅           
  ▄▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▁▄▁▄▁▁▁▁▁██▇█▄▇▅▅▅▅▇███▄▇▁▅▄▁▁▁▅ ▁
  56.6 ms         Histogram: frequency by time        76.1 ms <

 Memory estimate: 216.61 MiB, allocs estimate: 77293.

In [7]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

BenchmarkTools.Trial: 77 samples with 1 evaluation.
 Range (min … max):  60.073 ms … 73.338 ms  ┊ GC (min … max): 24.68% … 28.63%
 Time  (median):     64.967 ms              ┊ GC (median):    29.03%
 Time  (mean ± σ):   65.435 ms ±  2.355 ms  ┊ GC (mean ± σ):  28.75% ±  1.15%

                   ▅▂  █ ▅▁                                    
  ▃▁▃▁▁▅▁▁▁▁▁▁▁▁▁▅████▆████▁▃▁▅▃▅▅▃▁▁▁▃▁▁▅▁▃▃▁▁▃▃▃▁▁▁▁▁▁▁▁▁▁▃ ▁
  60.1 ms         Histogram: frequency by time        73.3 ms <

 Memory estimate: 215.77 MiB, allocs estimate: 76912.

In [8]:
Conv_backward_v1(input, weights, bias, gradient) == Conv_backward_v2(input, weights, bias, gradient)

true